# Option Data
## Call & Put

In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize
figsize(10,6)
import time

import yfinance as yf
from yfinance.utils import get_json
from yfinance import download

from pandas_datareader.data import Options
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import numpy as np

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import bs4 as bs
from bs4 import BeautifulSoup
import requests

In [2]:
# Scrap sp500 tickers
def save_sp500_tickers():

    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'html')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        if not '.' in ticker:
            tickers.append(ticker.replace('\n',''))
        
    return tickers

symbols = save_sp500_tickers()

In [39]:
# ------------------------------------------------------------------------------
# run aggregation func
# ------------------------------------------------------------------------------
errors = []

call_dict = {}
put_dict = {}
date_dict = {}

for sym in tqdm(symbols[0:10]):
    print('-'*50)
    print('downloading {} ...'.format(sym))
    try:
        opt_sym = yf.Ticker(sym)
        date_dict[sym] = opt_sym.options
        date_long = opt_sym.options[-1]
        #date_short = opt_sym.options[1]
        opt = opt_sym.option_chain(date_long)
        call_dict[sym] = opt.calls
        put_dict[sym] = opt.puts
    except Exception as e:
        errors.append(sym)
        print('{} error: {}'.format(sym, e))
        continue
    else:
        print('{} complete'.format(sym))
        print()

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------
downloading MMM ...


 10%|█         | 1/10 [00:00<00:04,  2.14it/s]

MMM complete

--------------------------------------------------
downloading ABT ...


 20%|██        | 2/10 [00:00<00:03,  2.44it/s]

...

AMD complete

--------------------------------------------------
downloading AAP ...


 90%|█████████ | 9/10 [00:03<00:00,  2.41it/s]

AAP complete

--------------------------------------------------
downloading AES ...


100%|██████████| 10/10 [00:04<00:00,  2.39it/s]

AES complete



In [41]:
_option_data = '../../data/option_data/'

for sym in call_dict:
    opt_sym = yf.Ticker(sym)
    dates_dict[sym] = opt_sym.options
    date_long = opt_sym.options[4]
    tmp_ = calls_dict[sym]
    tmp_['symbol'] = sym
    tmp_['date'] = date_long
    df = tmp_[['symbol', 'date', 'contractSymbol', 'strike', 'lastPrice', 'bid', 'strike', 'ask', 'volume',  'impliedVolatility', 'inTheMoney']]
    df.columns = ['symbol','date','contract','strike', 'last',  'bid', 'strike', 'ask', 'volume', 'iv', 'in-the-money']
    df.to_excel(_option_data + "call/"+sym+".xlsx")

In [44]:
df[df["in-the-money"]==True]
#df[df["in-the-money"]==False]

,symbol,date,contract,strike,last,bid,strike,ask,volume,iv,in-the-money
0,AES,2022-01-21,AES230120C00010000,10.0000,14.8000,17.4000,10.0000,20.6000,1,0.8257,True
1,AES,2022-01-21,AES230120C00013000,13.0000,10.7400,13.8000,13.0000,18.2000,8,0.7373,True
2,AES,2022-01-21,AES230120C00015000,15.0000,8.8000,13.8000,15.0000,15.4000,2,0.5283,True
3,AES,2022-01-21,AES230120C00017000,17.0000,9.9100,12.5000,17.0000,13.0000,1,0.3999,True
4,AES,2022-01-21,AES230120C00020000,20.0000,7.9700,10.3000,20.0000,10.8000,3,0.3903,True
5,AES,2022-01-21,AES230120C00022000,22.0000,8.8000,9.0000,22.0000,9.5000,1,0.3860,True
6,AES,2022-01-21,AES230120C00025000,25.0000,7.4100,7.3000,25.0000,7.7000,8,0.3748,True
7,AES,2022-01-21,AES230120C00027000,27.0000,6.1000,6.2000,27.0000,6.8000,5,0.3784,True
